In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [5]:
def create_hotel_url(hotel_ref: str):
#     ref = hotel_ref.replace('.uk.html', '.ru.html')
    ref = hotel_ref
    url = f"https://www.booking.com/reviews/ua/hotel/{ref}?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=45a571d0756ef620ea794c3a3ec26fbb;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&"
    return url

In [48]:
def parse_topic_scores(hotel):
    hotel_url = create_hotel_url(hotel)
    result = requests.get(hotel_url)
    soup = BeautifulSoup(result.text, 'html.parser')
    review_score_list = soup.find("ul", class_="review_score_breakdown_list")
    clearfix_list = review_score_list.findAll("li", class_="clearfix")
    
    result = dict()
    for item_soup in clearfix_list:
        topic = item_soup.get("data-question")
        value_str = item_soup.findAll('p')[1].text
        value_str = value_str.replace(',', '.')
        result[topic] = float(value_str)

    return result

In [49]:
hotel_topics = {'hotel_clean', 'hotel_comfort', 'hotel_location', 'hotel_services', 'hotel_staff', 'hotel_value', 'hotel_free_wifi'}

In [60]:
def parse_topic_scores_for_hotels(hotels):
    result = {topic: [] for topic in  hotel_topics}
    result['hotel'] = []    
    for hotel in hotels:
        try:
            hotel_topic_scores = parse_topic_scores(hotel)
            result['hotel'].append(hotel)
            for topic in hotel_topics:
                if topic in hotel_topic_scores:
                    result[topic].append(hotel_topic_scores[topic])
                else:
                    result[topic].append(-1)
        except:
            continue
    
    
    return pd.DataFrame(result)

### Train data

In [73]:
df = pd.read_csv("booking-train.csv")

In [74]:
hotels = np.unique(df['hotel'].values)

In [63]:
%time hotel_topic_df = parse_topic_scores_for_hotels(hotels)

CPU times: user 1min 13s, sys: 1.04 s, total: 1min 14s
Wall time: 9min 1s


In [67]:
len(hotel_topic_df.loc[hotel_topic_df['hotel_free_wifi'] == -1.0])

46

In [68]:
hotel_topic_df

,hotel_comfort,hotel_services,hotel_free_wifi,hotel_clean,hotel_staff,hotel_value,hotel_location,hotel
0,8.0,8.1,7.3,8.5,8.9,8.6,8.9,10-rooms.uk.html
1,8.9,8.9,8.7,9.3,9.2,8.9,8.9,10rooms-apartments.uk.html
2,8.8,8.6,8.6,8.8,9.4,8.4,8.2,12-months-mini.uk.html
3,9.3,9.1,8.9,9.4,9.3,9.0,9.3,12-th-floor-odessa.uk.html
4,9.6,9.5,-1.0,10.0,10.0,9.5,9.6,2kh-komnatnaia-kvartira-odesa12.uk.html
...,...,...,...,...,...,...,...,...
537,5.1,5.3,6.2,5.7,6.6,6.0,7.8,zirka-odessa.uk.html
538,7.7,8.5,-1.0,8.8,9.1,8.3,7.9,zolotoe-runo-odesa.uk.html
539,8.8,8.3,-1.0,8.5,8.8,8.0,8.5,zory-the-guesthouse-odesa1.uk.html
540,9.1,9.0,8.8,9.0,9.5,9.0,7.9,zs-club.uk.html


In [70]:
hotel_topic_df.to_csv('hotel-topic-marks-train.csv')

### Test data

In [75]:
df_test = pd.read_csv("booking-test.csv")

In [76]:
hotels_test = np.unique(df_test['hotel'].values)

In [77]:
%time hotel_topic_df_test = parse_topic_scores_for_hotels(hotels_test)

CPU times: user 28.4 s, sys: 283 ms, total: 28.7 s
Wall time: 3min 22s


In [78]:
hotel_topic_df_test.to_csv('hotel-topic-marks-test.csv')

### Validation data

In [79]:
df_val = pd.read_csv("booking-val.csv")

In [80]:
hotels_val = np.unique(df_val['hotel'].values)

In [81]:
%time hotel_topic_df_val = parse_topic_scores_for_hotels(hotels_val)

CPU times: user 26.2 s, sys: 322 ms, total: 26.6 s
Wall time: 3min 8s


In [83]:
hotel_topic_df_val.to_csv('hotel-topic-marks-val.csv')